In [127]:
import requests
from bs4 import BeautifulSoup
import io
import os
import re
import pandas as pd

# 1. PDF, HWP 파일 저장하기

In [128]:
title_list = []
li_list = []
for i in range(1,3):
  url = 'https://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761'
  query ={'searchCnd': '1',
          'sdate': '2005-05-01',
          'edate': '2017-12-31',
          'pageIndex': i}
  resp = requests.get(url,query)
  soup = BeautifulSoup(resp.content,'lxml')
  li = soup.find('div', class_='bdLine type2').find('ul').find('li')
  li_li = li.find_next_siblings('li')
  li_list.append(li)
  li_list.extend(li_li)  # len(li_list) =20 brings 10 pages for each page when the loop is 2, 
  
  for j in range(0,len(li_list)):
        link = li_list[j].find('div', class_='fileGoupBox').find('ul').find('li').find('a').attrs['href']
        file_title = li_list[j].find('div', class_='fileGoupBox').find('ul').find('li').find('a').text
        file_extension = re.findall('([a-z]+)', file_title)
        title = li_list[j].find('div',class_='row').find('span').find('a').find('span').find('span').text
        
        link_url = 'https://www.bok.or.kr' + link
        pdf_file = requests.get(link_url)
        if file_extension == ['pdf']:
            title_list.append(title)
            with open('{}.pdf'.format(title),'wb') as f:
                f.write(pdf_file.content)
        else:
            with open('{}.hwp'.format(title), 'wb') as f:
                f.write(pdf_file.content)
            
    
#     except:
#       link2 = li_list[j].find('div').find('div').find('a').attrs['href']
#       title = li_list[j].find('div',class_='row').find('span').find('a').find('span').find('span').text
#       link_url2 = 'https://www.bok.or.kr' + link2
#       pdf_file2 = requests.get(link_url2)
#       with open('{}.hwp'.format(title), 'wb') as f:
#         f.write(pdf_file2.content)



In [120]:
title_list = []
li_list = []
x=[]
for i in range(1,3):
  url = 'https://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761'
  query ={'searchCnd': '1',
          'sdate': '2005-05-01',
          'edate': '2017-12-31',
          'pageIndex': i}
  resp = requests.get(url,query)
  soup = BeautifulSoup(resp.content,'lxml')
  li = soup.find('div', class_='bdLine type2').find('ul').find('li')
  li_li = li.find_next_siblings('li')
  li_list.append(li)
  li_list.extend(li_li)  # len(li_list) =20 brings 10 pages for each page when the loop is 2, 
  
  for j in range(0,len(li_list)):
        link = li_list[j].find('div', class_='fileGoupBox').find('ul').find('li').find('a').text
        file_name = re.findall('([a-z]+)', link)
        print(file_name)


['pdf']
['pdf']
['pdf']
['hwp']
['pdf']
['pdf']
['pdf']
['pdf']
['pdf']
['pdf']
['pdf']
['pdf']
['pdf']
['hwp']
['pdf']
['pdf']
['pdf']
['pdf']
['pdf']
['pdf']
['pdf']
['hwp']
['hwp']
['hwp']
['pdf']
['pdf']
['pdf']
['pdf']
['pdf']
['pdf']


In [124]:
len(title_list)

25

# 2. 파일 Text 추출

In [122]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
        text = retstr.getvalue()
    fp.close()
    device.close()
    retstr.close()
    return text

In [192]:
result={}
for i in title_list:
    try:
        result[i]=convert_pdf_to_txt('{}.pdf'.format(i))
    except:
        print(i)

In [193]:
for key,value in result.items():
    result[key].replace('\t', '').replace('\n', '').replace('\x0c', '')
result

{'금융통화위원회 의사록(2017년도 제23차)(2017.12.7.)': '2017년도 제23차\n\n금융통화위원회(정기) 의사록\n\n한 국 은 행\n\n\x0c1. 일\n\n시\n\n2017년 12월 7일(목)\n\n2. 장\n\n소\n\n금융통화위원회 회의실\n\n3. 출석위원\n\n이 주 열 의 장(총재)\n\n함 준 호 위 원\n\n이 일 형 위 원\n\n조 동 철 위 원\n\n고 승 범 위 원\n\n신 인 석 위 원\n\n윤 면 식 위 원(부총재)\n\n4. 결석위원\n\n없 음\n\n5. 참 여 자\n\n하 성 감 사\n\n임 형 준 부총재보\n\n전 승 철 부총재보\n\n장 민 조사국장\n\n김 민 호 부총재보\n\n허 진 호 부총재보\n\n서 봉 국 외자운용원장\n\n손\n\n욱 경제연구원장\n\n신 운 금융안정국장\n\n박 종 석 통화정책국장\n\n이 환 석 금융시장국장\n\n차 현 진 금융결제국장\n\n이 승 헌 국제국장\n\n성 병 희 공보관\n\n장 정 석 금융통화위원회실장\n\n이 동 원 의사팀장\n\n6. 회의경과\n\n<의안 제41호 - 한국은행의 금융기관대출규정 개정(안)>\n\n(１) 담당 부총재보가 한국은행법 제28조 제3항 및 제64조 제1항에 의거 한\n\n국은행의 금융기관에 대한 대출의 적격담보에 한국주택금융공사 발행 주택저당증\n\n권, 즉 MBS를 포함하는 기간을 연장하고자 한다는 내용의 제안설명을 하였음.\n\n(２) 위원 토의내용\n\n다음과 같이 보고하였음.\n\n제안설명에 이어 관련부서는 본 안건에 대해 위원협의회에서 논의된 내용을\n\n- 1 -\n\n\x0c먼저 위원들은 안심전환대출 취급으로 MBS를 보유하게 된 은행들의 부담이\n\n해소되지 않은 점 등을 고려하여 지난해에 MBS의 담보증권 인정기한을 1년 연장\n\n한 바 있는데 내년에도 은행의 MBS 보유부담이 지속될 것인지 그리고 담보증권\n\n인정 기한을 1년 더 연장하고 나면 동 부담이 해소되는지에 대해 물었으며,\n\n이에 대해 관련부서에서는 국

In [190]:
for key,value in result.items():
    result[key]=re.findall('[가-힣]+',value)
pd.DataFrame(result.items(), columns=['Title','Content'])

,Title,Content
0,금융통화위원회 의사록(2017년도 제23차)(2017.12.7.),"[년도, 제, 차, 금융통화위원회, 정기, 의사록, 한, 국, 은, 행, 일, 시,..."
1,금융통화위원회 의사록(2017년도 제22차)(2017.11.30.),"[년도, 제, 차, 금융통화위원회, 정기, 의사록, 한, 국, 은, 행, 일, 장,..."
2,금융통화위원회 의사록(2017년도 제21차)(2017.11.9.),"[년도, 제, 차, 금융통화위원회, 정기, 의사록, 한, 국, 은, 행, 일, 시,..."
3,금융통화위원회 의사록(2017년도 제18차)(2017.9.21.),"[년도, 제, 차, 금융통화위원회, 정기, 의사록, 한, 국, 은, 행, 일, 시,..."
4,금융통화위원회 의사록(2017년도 제16차)(2017.8.31.),"[년도, 제, 차, 금융통화위원회, 정기, 의사록, 한, 국, 은, 행, 일, 장,..."
5,금융통화위원회 의사록(2017년도 제15차)(2017.8.10.),"[년도, 제, 차, 금융통화위원회, 정기, 의사록, 한, 국, 은, 행, 일, 시,..."
6,금융통화위원회 의사록(2017년도 제13차)(2017.7.13.),"[년도, 제, 차, 금융통화위원회, 정기, 의사록, 한, 국, 은, 행, 일, 장,..."
7,금융통화위원회 의사록(2017년도 제12차)(2017.6.22.),"[년도, 제, 차, 금융통화위원회, 정기, 의사록, 한, 국, 은, 행, 일, 자,..."
8,금융통화위원회 의사록(2017년도 제10차)(2017.5.25.),"[년도, 제, 차, 금융통화위원회, 정기, 의사록, 한, 국, 은, 행, 일, 장,..."
9,금융통화위원회 의사록(2017년도 제8차)(2017.4.27.),"[년도, 제, 차, 금융통화위원회, 정기, 의사록, 한, 국, 은, 행, 일, 시,..."


In [137]:
ddd=pd.DataFrame(result.items(), columns=['Title','Content'])

In [171]:
g=ddd[['Content']].iloc[0].values

In [174]:
list(g)[0].replace('\t', '').replace('\n', '').replace('\x0c', '')

'2017년도 제23차금융통화위원회(정기) 의사록한 국 은 행1. 일시2017년 12월 7일(목)2. 장소금융통화위원회 회의실3. 출석위원이 주 열 의 장(총재)함 준 호 위 원이 일 형 위 원조 동 철 위 원고 승 범 위 원신 인 석 위 원윤 면 식 위 원(부총재)4. 결석위원없 음5. 참 여 자하 성 감 사임 형 준 부총재보전 승 철 부총재보장 민 조사국장김 민 호 부총재보허 진 호 부총재보서 봉 국 외자운용원장손욱 경제연구원장신 운 금융안정국장박 종 석 통화정책국장이 환 석 금융시장국장차 현 진 금융결제국장이 승 헌 국제국장성 병 희 공보관장 정 석 금융통화위원회실장이 동 원 의사팀장6. 회의경과<의안 제41호 - 한국은행의 금융기관대출규정 개정(안)>(１) 담당 부총재보가 한국은행법 제28조 제3항 및 제64조 제1항에 의거 한국은행의 금융기관에 대한 대출의 적격담보에 한국주택금융공사 발행 주택저당증권, 즉 MBS를 포함하는 기간을 연장하고자 한다는 내용의 제안설명을 하였음.(２) 위원 토의내용다음과 같이 보고하였음.제안설명에 이어 관련부서는 본 안건에 대해 위원협의회에서 논의된 내용을- 1 -먼저 위원들은 안심전환대출 취급으로 MBS를 보유하게 된 은행들의 부담이해소되지 않은 점 등을 고려하여 지난해에 MBS의 담보증권 인정기한을 1년 연장한 바 있는데 내년에도 은행의 MBS 보유부담이 지속될 것인지 그리고 담보증권인정 기한을 1년 더 연장하고 나면 동 부담이 해소되는지에 대해 물었으며,이에 대해 관련부서에서는 국채 등에 비해 MBS의 유통성이 낮기 때문에 내년에도 보유부담이 지속될 것으로 보이지만, 2015년 5월에서 7월중 발행된 안심전환대출 관련 MBS의 경우 평균 듀레이션을 감안할 때 2018년 말이 되면 보유부담이 거의 해소될 것으로 보인다고 답변하였음.또한 일부 위원들은 MBS 보유부담이 2018년 말에 해소될 것으로 보인다면담보증권 인정기한을 1년 더 연장하되 담보증권으로 인정하는 한도는 현행 30%보다 낮은 수준으로 변경하는 것이 어떠한

In [153]:
gg=re.findall('[가-힣]+',g[0])

In [154]:
gg[:5]

['년도', '제', '차', '금융통화위원회', '정기']

In [155]:
rl=''
for gf in gg:
    rl+=gf+' '

In [156]:
rl

'년도 제 차 금융통화위원회 정기 의사록 한 국 은 행 일 시 년 월 일 목 장 소 금융통화위원회 회의실 출석위원 이 주 열 의 장 총재 함 준 호 위 원 이 일 형 위 원 조 동 철 위 원 고 승 범 위 원 신 인 석 위 원 윤 면 식 위 원 부총재 결석위원 없 음 참 여 자 하 성 감 사 임 형 준 부총재보 전 승 철 부총재보 장 민 조사국장 김 민 호 부총재보 허 진 호 부총재보 서 봉 국 외자운용원장 손 욱 경제연구원장 신 운 금융안정국장 박 종 석 통화정책국장 이 환 석 금융시장국장 차 현 진 금융결제국장 이 승 헌 국제국장 성 병 희 공보관 장 정 석 금융통화위원회실장 이 동 원 의사팀장 회의경과 의안 제 호 한국은행의 금융기관대출규정 개정 안 담당 부총재보가 한국은행법 제 조 제 항 및 제 조 제 항에 의거 한 국은행의 금융기관에 대한 대출의 적격담보에 한국주택금융공사 발행 주택저당증 권 즉 를 포함하는 기간을 연장하고자 한다는 내용의 제안설명을 하였음 위원 토의내용 다음과 같이 보고하였음 제안설명에 이어 관련부서는 본 안건에 대해 위원협의회에서 논의된 내용을 먼저 위원들은 안심전환대출 취급으로 를 보유하게 된 은행들의 부담이 해소되지 않은 점 등을 고려하여 지난해에 의 담보증권 인정기한을 년 연장 한 바 있는데 내년에도 은행의 보유부담이 지속될 것인지 그리고 담보증권 인정 기한을 년 더 연장하고 나면 동 부담이 해소되는지에 대해 물었으며 이에 대해 관련부서에서는 국채 등에 비해 의 유통성이 낮기 때문에 내 년에도 보유부담이 지속될 것으로 보이지만 년 월에서 월중 발행된 안심 전환대출 관련 의 경우 평균 듀레이션을 감안할 때 년 말이 되면 보유부 담이 거의 해소될 것으로 보인다고 답변하였음 또한 일부 위원들은 보유부담이 년 말에 해소될 것으로 보인다면 담보증권 인정기한을 년 더 연장하되 담보증권으로 인정하는 한도는 현행 보다 낮은 수준으로 변경하는 것이 어떠한지 물어보고 이 같이 한도를 낮출 경 우 한국은행이 년 말에는 담보인정 조치를 예정대로 종